In [1]:
# Init
import os
import sys

module_path = os.path.abspath(os.path.join("../src/simulicronalpha/"))
if module_path not in sys.path:
    sys.path.append(module_path)

# Imports
import numpy as np
import pandas as pd
import warnings
import multiprocessing
np.set_printoptions(suppress=True)
from numpy import concatenate as c
from numpy import cumsum
import random
%load_ext line_profiler

In [2]:
from generateSim import (generatePopulation, generateGenome)
from stats import stats
from regulation import regulation
from checkCopyNumber import checkCopyNumber
from transposition import transposition
#from recombination import recombination

In [3]:
from popSim import runBatch, runSim

In [4]:
gen,piset,piIndice, rates = generateGenome(
    numberOfInsertionSites=10000,
    numberOfChromosomes=6,
    baseRecombinationRate=0.1,
    baseSelection=0,
    baseTau=1,
)
pop, tr , TEset= generatePopulation(
    gen,
    piIndice,
    NumberOfIndividual=1000,
    NumberOfTransposonTypes=1,
    NumberOfInsertionsPerType=[1],
    FrequencyOfInsertions=[1.0],
    ExcisionRates = [1.0]
)

In [5]:
%lprun -f transposition runSim(gen,pop,tr,TEset,1,5,rates,piset)

0
1
2
3
4


Timer unit: 1e-06 s

Total time: 3.76629 s
File: /home/siddharth/Documents/Projects/Simulicron/src/simulicronalpha/transposition.py
Function: transposition at line 8

Line #      Hits         Time  Per Hit   % Time  Line Contents
     8                                           def transposition(
     9                                               transposonMatrix,
    10                                               genomeMatrix,
    11                                               NumberOfTransposonInsertions,
    12                                               TEset,
    13                                               piCoord,
    14                                               v1,
    15                                               v2,
    16                                           ):
    17                                               # If any of the haplotyle is empty, convert the appr.
    18                                               # parameters to empty lists
    1

In [7]:
runSim(gen,pop,tr,TEset,1,10,rates,piset)

0
1
2
3
4
5
6
7
8
9


{'State': 'FLUX',
 'Generatrion': 11,
 'NTE': 101063.0,
 'AvgCopyNum': [1.0,
  1.899,
  3.584,
  6.821,
  11.69,
  19.418,
  29.419,
  40.94,
  50.637,
  59.982,
  69.077],
 'CopyNumVar': [0.984,
  3.516799,
  9.868943999999999,
  26.568958999999996,
  54.0879,
  129.33727599999997,
  287.92343900000003,
  584.2304,
  813.4132309999999,
  1007.8736759999999,
  1149.599071]}

In [ ]:
%lprun -f recombination recombination(rates=rates, transposonMatrix=tr, v1=[52], v2=[65])

In [ ]:
%lprun -f runBatch runBatch(numberOfSimulations=1,baseSelection=0,baseInsertionProb=1,numberOfInsertionSites=10000,numberOfChromosomes=6,baseRecombinationRate=0.01,baseTau=1,numberOfPiRNA=6,piPercentage=3,enablePiRecombination=False,NumberOfIndividual=1000,NumberOfTransposonTypes=1,NumberOfInsertionsPerType=[1],FrequencyOfInsertions=[1.0],ExcisionRates=[1.0],RepairRates=[1],InsertionRates=[1],HardyWeinberg=False,NumberOfGenerations=10,numberOfThreads=1,)

In [ ]:
np.set_printoptions(suppress=True,)

In [ ]:
%lprun -f transposition transposition(transposonMatrix=tr, genomeMatrix=gen, NumberOfTransposonInsertions=1, TEset=TEset, v1=[52,101,321,433,12,23,234,123,657,29], v2=[65,54,22,121,23])

In [ ]:
piIndice[1]

In [5]:
piset

{1: (372, 422),
 2: (2144, 2194),
 3: (3686, 3736),
 4: (6212, 6262),
 5: (8248, 8298),
 6: (8704, 8754)}

In [6]:
list(range(1,4))

[1, 2, 3]

In [16]:
piCoord = []
for i in piset.values():
    piCoord = piCoord + (list(range(i[0],i[1])))

False

In [30]:
(tr[[2,3],3] * 0).tolist()

[0.0, 0.0]

In [24]:
list(TEset.keys())

[1]

In [25]:
for i in TEset.keys():
    print (i)

1


In [26]:
len(TEset.keys())

1

In [11]:
tr

array([[   0.,    0.,    0.,    0.,    0.,    0.],
       [   0., 1950.,    0.,    1.,    1.,    1.],
       [   0., 4238.,    0.,    1.,    1.,    1.],
       ...,
       [   0., 7038.,    0.,    1.,    1.,    1.],
       [   0., 8479.,    0.,    1.,    1.,    1.],
       [   0., 7056.,    0.,    1.,    1.,    1.]])

In [18]:
k = [0.0,1.0,1.0,1.0,1.0,1.0]

In [21]:
np.append(tr, np.array([k]), axis=0)

array([[   0.,    0.,    0.,    0.,    0.,    0.],
       [   0., 1950.,    0.,    1.,    1.,    1.],
       [   0., 4238.,    0.,    1.,    1.,    1.],
       ...,
       [   0., 8479.,    0.,    1.,    1.,    1.],
       [   0., 7056.,    0.,    1.,    1.,    1.],
       [   0.,    1.,    1.,    1.,    1.,    1.]])

In [22]:
tr

array([[   0.,    0.,    0.,    0.,    0.,    0.],
       [   0., 1950.,    0.,    1.,    1.,    1.],
       [   0., 4238.,    0.,    1.,    1.,    1.],
       ...,
       [   0., 7038.,    0.,    1.,    1.,    1.],
       [   0., 8479.,    0.,    1.,    1.,    1.],
       [   0., 7056.,    0.,    1.,    1.,    1.]])